In [1]:
import gym
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, BatchNormalization

Using TensorFlow backend.


In [2]:
import gym
import random
import numpy as np


LR = 1e-3
env = gym.make("CartPole-v0")
env.reset()
goal_steps = 500
score_requirement = 65
initial_games = 100000

In [3]:
def some_random_games_first():
    # Each of these is its own game.
    for episode in range(10):
        env.reset()
        # this is each frame, up to 200...but we wont make it that far.
        for t in range(20):
            # This will display the environment
            # Only display if you really want to see it.
            # Takes much longer to display it.
            env.render()
            
            # This will just create a sample action in any environment.
            # In this environment, the action can be 0 or 1, which is left or right
            action = env.action_space.sample()
            
            # this executes the environment with an action, 
            # and returns the observation of the environment, 
            # the reward, if the env is over, and other info.
            observation, reward, done, info = env.step(action)
            if done:
                break
    env.close()
                
some_random_games_first()

In [6]:
def initial_population():
    # x for 
    x_train = []
    y_train = []
    # all scores:
    scores = []
    # just the scores that met our threshold:
    accepted_scores = []
    # iterate through however many games we want:
    for _ in range(initial_games):
        score = 0
        
        # moves specifically from this environment:
        game_memory = []
        # previous observation that we saw
        prev_observation = []
        # for each frame in 200
        for _ in range(goal_steps):
            # choose random action (0 or 1)
            action = random.randrange(0,2)
            # do it!
            observation, reward, done, info = env.step(action)
            
            # notice that the observation is returned FROM the action
            # so we'll store the previous observation here, pairing
            # the prev observation to the action we'll take.
            if len(prev_observation) > 0 :
                game_memory.append([prev_observation, action])
            prev_observation = observation
            score+=reward
            if done:
                env.reset()
                break

        # IF our score is higher than our threshold, we'd like to save
        # every move we made
        # NOTE the reinforcement methodology here. 
        # all we're doing is reinforcing the score, we're not trying 
        # to influence the machine in any way as to HOW that score is 
        # reached.
        if score >= score_requirement:
            accepted_scores.append(score)
            for data in game_memory:
                # convert to one-hot (this is the output layer for our neural network)
                if data[1] == 1:
                    output = [0,1]
                elif data[1] == 0:
                    output = [1,0]
                    
                # saving our training data
                x_train.append(data[0])
                y_train.append(output)

        # reset env to play again
        
        # save overall scores
        scores.append(score)
    
    # just in case you wanted to reference later
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    np.save('saved x_train.npy',x_train)
    np.save('saved y_train.npy',y_train)
    
    # some stats here, to further illustrate the neural network magic!
    print('Average accepted score:',np.mean(accepted_scores))
    print('Median score for accepted scores:',np.median(accepted_scores))
    print(len(accepted_scores))
    
    return x_train, y_train

In [8]:
x_train, y_train = initial_population()


Average accepted score: 76.02102102102103
Median score for accepted scores: 73.0
999


In [9]:
x_train.shape
y_train.shape

(74946, 2)

In [10]:
def build_NN():
    m = Sequential()
    
    m.add(Dense(128, input_dim=4))
    m.add(Activation('relu'))
    m.add(BatchNormalization())
    m.add(Dropout(0.2))
    
    m.add(Dense(128))
    m.add(Activation('relu'))
    m.add(BatchNormalization())
    
    m.add(Dense(128))
    m.add(Activation('relu'))
    m.add(BatchNormalization())
    
    m.add(Dense(128))
    m.add(Activation('relu'))
    m.add(BatchNormalization())
    m.add(Dropout(0.2))
    
    m.add(Dense(128))
    m.add(Activation('relu'))
    m.add(BatchNormalization())
    
    m.add(Dense(128))
    m.add(Activation('relu'))
    m.add(BatchNormalization())
    m.add(Dropout(0.2))
    
    m.add(Dense(64))
    m.add(Activation('relu'))
    m.add(BatchNormalization())
    m.add(Dropout(0.2))
    
    m.add(Dense(32))
    m.add(Activation('relu'))
    m.add(BatchNormalization())
    m.add(Dropout(0.2))
    
    m.add(Dense(2))
    m.add(Activation('softmax'))
    
    m.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
    
    m.summary()
    return m
m = build_NN()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 128)               640       
_________________________________________________________________
activation_8 (Activation)    (None, 128)               0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 128)               512       
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)               16512     
_________________________________________________________________
activation_9 (Activation)    (None, 128)               0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 128)               512       
__________

In [11]:
m.fit(x_train, y_train, epochs=30, batch_size=32, validation_split=0.1)

Instructions for updating:
Use tf.cast instead.
Train on 67451 samples, validate on 7495 samples
Epoch 1/30
67451/67451 [==============================] - 19s 278us/step - loss: 0.6794 - acc: 0.5917 - val_loss: 0.6551 - val_acc: 0.6041
Epoch 2/30
67451/67451 [==============================] - 15s 219us/step - loss: 0.6619 - acc: 0.6062 - val_loss: 0.6559 - val_acc: 0.6025
Epoch 3/30
67451/67451 [==============================] - 15s 219us/step - loss: 0.6595 - acc: 0.6049 - val_loss: 0.6549 - val_acc: 0.6049
Epoch 4/30
67451/67451 [==============================] - 15s 219us/step - loss: 0.6579 - acc: 0.6070 - val_loss: 0.6511 - val_acc: 0.6075
Epoch 5/30
67451/67451 [==============================] - 15s 219us/step - loss: 0.6570 - acc: 0.6067 - val_loss: 0.6513 - val_acc: 0.6025
Epoch 6/30
67451/67451 [==============================] - 15s 221us/step - loss: 0.6565 - acc: 0.6090 - val_loss: 0.6508 - val_acc: 0.6143
Epoch 7/30
67451/67451 [==============================] - 15s 222us/s

In [12]:
scores = []
choices = []
x_train = []
y_train = []
for each_game in range(1000):
    if each_game % 50 ==0 and len(scores)!=0:
        print('current game:',each_game, 'satisfied games:', len(scores))
        print('Average Score:',sum(scores)/len(scores))
        print('highest:', max(scores))
        print('lowest:', min(scores))
        print('choice 1:{}  choice 0:{}'.format(choices.count(1)/len(choices),choices.count(0)/len(choices)))
        score_requirement += 5
    score = 0
    game_memory = []
    prev_obs = []
    env.reset()
    for _ in range(goal_steps):
        if each_game % 40 == 0:
            env.render()

        if len(prev_obs)==0:
            action = random.randrange(0,2)
        else:
            action = np.argmax(m.predict(prev_obs.reshape(1,4),batch_size=1 ))

        choices.append(action)
                
        new_observation, reward, done, info = env.step(action)
        prev_obs = new_observation
        game_memory.append([new_observation, action])
        score+=reward
        
        if done:
            break
    if score >= score_requirement * 1.3:
        for data in game_memory:
            # convert to one-hot (this is the output layer for our neural network)
            if data[1] == 1:
                output = [0,1]
            elif data[1] == 0:
                output = [1,0]

            # saving our training data
            x_train.append(data[0])
            y_train.append(output)

        # save overall scores
        scores.append(score)

x_train = np.array(x_train)
y_train = np.array(y_train)
np.save('saved x_train.npy',x_train)
np.save('saved y_train.npy',y_train)
    
print('Average Score:',sum(scores)/len(scores))
print('highest:', max(scores))
print('lowest:', min(scores))
print('choice 1:{}  choice 0:{}'.format(choices.count(1)/len(choices),choices.count(0)/len(choices)))
print(score_requirement)

current game: 50 satisfied games: 50
Average Score: 161.04
highest: 200.0
lowest: 99.0
choice 1:0.5091902632886239  choice 0:0.49080973671137607
current game: 100 satisfied games: 100
Average Score: 158.57
highest: 200.0
lowest: 95.0
choice 1:0.5097433310210002  choice 0:0.49025666897899983
current game: 150 satisfied games: 150
Average Score: 162.52666666666667
highest: 200.0
lowest: 95.0
choice 1:0.5092907830509865  choice 0:0.4907092169490135
current game: 200 satisfied games: 199
Average Score: 164.3718592964824
highest: 200.0
lowest: 95.0
choice 1:0.5089446256057051  choice 0:0.4910553743942949
current game: 250 satisfied games: 246
Average Score: 166.890243902439
highest: 200.0
lowest: 95.0
choice 1:0.5084995057025052  choice 0:0.49150049429749476
current game: 300 satisfied games: 287
Average Score: 167.14982578397212
highest: 200.0
lowest: 95.0
choice 1:0.5089560706759605  choice 0:0.49104392932403956
current game: 350 satisfied games: 330
Average Score: 168.66363636363636
high